<a href="https://colab.research.google.com/github/javierdealba/Notes/blob/main/MNA_IAyAA_semana_10_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

- Andre Nicolai Gutiérrez Bautista A01793606
- Javier Munoz Barrios A01794423
- Fernando Ivan Ramos Esparza A01373377
- Javier de Alba Pérez A01226046


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

Se trabajará con el conjunto de datos llamado "Restaurant & consumer", para la generación de un 
sistema de recomendación. Estos sistemas están dentro de los llamados métodos no supervisados y que 
se basan en técnicas de reducción de dimensionalidad. 
El archivo a descargar se llama RCdata.zip y se encuentra en la siguiente liga del repositorio de la UCI: 

https://archive.ics.uci.edu/ml/datasets/Restaurant+%26+consumer+data  
En particular se trabajará con los siguientes archvios, los cuales contienen la información mínima que 
requerimos para esta actividad: rating_final.csv y geoplaces2.csv. 

El objetivo es diseñar tres sistemas de recomendación basados en la evaluación de los usarios a los 
restaurantes. Dicha evaluación será la general, la referente al servicio y la referente a la comida. 

In [ ]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

#SVD
from sklearn.decomposition import TruncatedSVD

In [ ]:
def download_and_unzip(url, extract_to='.'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)

#**Ejercicio-1.** 

Descarga los archivos y genera la matriz de utilidad en la cual los renglones son los usuarios 
(user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones 
generales dadas por los usuarios (rating). En particular indica el tamaño de dicha matriz y despliega 
sus primeros 10 renglones. 

In [ ]:
zip_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'
download_and_unzip(zip_url,'./')

In [ ]:
PLACES_URL_PATH = './geoplaces2.csv'
data_places = pd.read_csv(PLACES_URL_PATH,encoding="latin-1")
dfPlaces = data_places.copy()
dfPlaces.columns

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services'],
      dtype='object')

In [ ]:
URL_PATH = './rating_final.csv'
data = pd.read_csv(URL_PATH)
df = data.copy()
df

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2
...,...,...,...,...,...
1156,U1043,132630,1,1,1
1157,U1011,132715,1,1,0
1158,U1068,132733,1,1,0
1159,U1068,132594,1,1,1


In [ ]:
tabla_relacional = pd.merge(df,dfPlaces,on='placeID')  
tabla_relacional.head()                      

,userID,placeID,rating,food_rating,service_rating,latitude,longitude,the_geom_meter,name,address,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,U1077,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,U1108,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,U1081,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,U1056,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,U1134,135085,2,1,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none


In [ ]:
tabla_relacional.shape

(1161, 25)

In [ ]:
UtMx = tabla_relacional.pivot_table(values='rating', index='userID',columns='name', fill_value=0)
UtMx.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
UtMx.shape 

(138, 129)

In [ ]:
print('Total de elementos de la matriz de utilidad: %d' %(UtMx.size))
print('Total de elementos diferentes de cero: %d' %(np.count_nonzero(UtMx)))
print('Procentaje de elementos diferentes de cero: %.1f%%'%(100*np.count_nonzero(UtMx) / UtMx.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 907
Procentaje de elementos diferentes de cero: 5.1%


#**Ejercicio-2.**

Con el objetivo de obtener la información con respecto a los vectores latentes de los restaurantes, 
obtener dicha factorización SVD utilizando todas las componentes posibles, es decir, usando todos 
sus valores singulares. En particular indica la cantidad de valores singulares obtenidos. 

In [ ]:
X = UtMx.T # Usamos la Transpuesta para cambiar los renglones
X.shape

(129, 138)

In [ ]:
# el numero de componentes debe ser menor que el numero de caracteristicas / usuarios
SVD = TruncatedSVD(n_components=137,random_state=1)
SVD.fit(X)
num_sv = 10 # 10
print('Cantidad de informacion simplificada de los primeros %d vectores singulares:'% num_sv)
print('%.1f%%' % (100 * (1-(SVD.singular_values_[0:num_sv]).sum()/(SVD.singular_values_).sum())))

Cantidad de informacion simplificada de los primeros 10 vectores singulares:
75.6%


In [ ]:
SVD.singular_values_

array([1.89824556e+01, 1.23895348e+01, 9.71004954e+00, 9.21987252e+00,
       9.19166417e+00, 8.83562133e+00, 8.35490731e+00, 8.16269046e+00,
       7.67589308e+00, 7.53252900e+00, 7.24693828e+00, 7.03772875e+00,
       6.96568920e+00, 6.94921261e+00, 6.72377310e+00, 6.62729356e+00,
       6.42888935e+00, 6.23122752e+00, 5.98488156e+00, 5.84593442e+00,
       5.52931996e+00, 5.45741148e+00, 5.43575015e+00, 5.30313453e+00,
       5.23195921e+00, 5.07778209e+00, 4.92772427e+00, 4.80719927e+00,
       4.77806009e+00, 4.72042746e+00, 4.68168774e+00, 4.48126959e+00,
       4.47285668e+00, 4.41292142e+00, 4.38371497e+00, 4.36866765e+00,
       4.34607925e+00, 4.19914184e+00, 4.07007621e+00, 4.04670751e+00,
       3.91390069e+00, 3.90244894e+00, 3.86052461e+00, 3.85945223e+00,
       3.79499753e+00, 3.62195823e+00, 3.62016459e+00, 3.50096878e+00,
       3.41779782e+00, 3.34322000e+00, 3.30294415e+00, 3.16880603e+00,
       3.14312670e+00, 3.11631317e+00, 3.07616268e+00, 2.95816128e+00,
      

#**Ejercicio-3.**

Usando la métrica de correlación de Pearon, obtener las 10 mejores recomendaciones de este 
modelo no supervisado con base a la información de alguien que evaluó bien de manera general al 
“Subway”. Despliega también el valor numérico de la correlación de Pearson de cada una de estas 10 
recomendaciones. NOTA: No debe estar incluido “Subway” dentro de las recomendaciones.

In [ ]:
num_sv = 10
SVD = TruncatedSVD(n_components=num_sv,random_state=1)

resultant_matrix = SVD.fit_transform(X)
resultant_matrix

array([[ 2.23152926e-01, -3.35116866e-01,  9.25838896e-01, ...,
        -4.25164536e-01,  1.35520819e-01,  9.11462273e-02],
       [ 6.35792462e-07, -3.06079229e-05, -1.89178020e-04, ...,
         5.32359105e-03, -8.36912474e-03,  1.77517017e-02],
       [ 3.04948289e+00,  2.56482771e+00, -4.78954246e-01, ...,
         8.62860382e-01,  5.63275120e-01, -1.63172328e+00],
       ...,
       [ 8.68871575e-01,  1.45553077e+00,  1.81301146e+00, ...,
         8.15336665e-01, -1.00887373e+00, -5.61747380e-01],
       [-1.27510648e-07,  2.03157159e-05,  2.80594385e-04, ...,
        -3.53970252e-03,  8.36839944e-03, -1.28839773e-02],
       [-3.47565950e-07,  2.77022124e-05,  6.80898323e-04, ...,
        -3.72045676e-03,  1.96057585e-02, -4.19379414e-02]])

In [ ]:
CorrMx = np.corrcoef(resultant_matrix)
CorrMx.shape

(129, 129)

In [ ]:
CorrMx[0:5,0:5]

array([[ 1.        , -0.18602401, -0.22452697, -0.15148281, -0.15895966],
       [-0.18602401,  1.        , -0.29054956, -0.11538374,  0.98731165],
       [-0.22452697, -0.29054956,  1.        ,  0.15029546, -0.20755881],
       [-0.15148281, -0.11538374,  0.15029546,  1.        , -0.15238433],
       [-0.15895966,  0.98731165, -0.20755881, -0.15238433,  1.        ]])

In [ ]:
restaurant = "Subway"
places = UtMx.columns 
places_list = list(places) # saber en que columna estaba Subway 
id_restaurant = places_list.index(restaurant) # busquemos el undice del restaurat Subway

id_restaurant
places

Index(['Abondance Restaurante Bar', 'Arrachela Grill', 'Cabana Huasteca',
       'Cafe Chaires', 'Cafeteria cenidet',
       'Cafeteria y Restaurant El Pacifico', 'Carls Jr',
       'Carnitas Mata  Calle 16 de Septiembre',
       'Carreton de Flautas y Migadas', 'Cenaduria El RincÃ³n de Tlaquepaque',
       ...
       'puesto de tacos', 'rockabilly ', 'shi ro ie', 'sirloin stockade',
       'tacos abi', 'tacos de barbacoa enfrente del Tec',
       'tacos de la estacion', 'tacos los volcanes', 'tortas hawai', 'vips'],
      dtype='object', name='name', length=129)

In [ ]:
def get_corr_recomendation(id):
  corr_recomienda = CorrMx[id]
  print(corr_recomienda.shape)
  print(corr_recomienda)
  return corr_recomienda

corr_recomienda = get_corr_recomendation(id_restaurant)

(129,)
[-0.15064635  0.97877781 -0.1845724  -0.15983695  0.99887223 -0.07193956
 -0.01573204  0.18193487  0.02371689 -0.0721253  -0.12088734  0.99990361
 -0.06747016 -0.12881113 -0.09992082 -0.01851576  0.99669844 -0.05222151
  0.06912151 -0.2865713   0.99326413  0.9972279  -0.02609229 -0.01136617
  0.09097883  0.03500756 -0.06497314 -0.01839213 -0.07070041  0.99712923
 -0.09078034  0.07195591  0.01535248 -0.14201129  0.07756104 -0.17304099
  0.05330264  0.99939672  0.99800799  0.04505749 -0.19152571  0.99990619
  0.99897094 -0.04660154 -0.01223637  0.99987903  0.99997694 -0.16809713
  0.05646127 -0.07411778 -0.15513466  0.99795453 -0.0428533  -0.02754784
  0.99737849  0.99911926 -0.06908106 -0.0535078   0.99995595 -0.37949259
  0.99992724  0.0133778  -0.09546762 -0.04008912  0.9987684  -0.12605574
 -0.07861828  0.02977735 -0.08655958 -0.23038202 -0.23325957  0.0014082
 -0.27000998 -0.09780463  0.05936832 -0.1409088  -0.15376939  0.02895601
  0.03207908 -0.13242192 -0.12764337  0.02587

In [ ]:
corr_recomienda[id_restaurant]

1.0

In [ ]:
idx = (corr_recomienda > .95)& (corr_recomienda < .999)
tmp = list()
for i in range(len(places[idx])):
  tmp.append((corr_recomienda[idx][i],places[idx][i]))

sorted(tmp, key=lambda x:x[0], reverse=True)

[(0.9989709443202156, 'McDonalds Centro'),
 (0.9988722342423437, 'Cafeteria cenidet'),
 (0.9987684032672002, 'Restaurant los Pinos'),
 (0.9980079912404478, 'Los Vikingos'),
 (0.9979545297320486, 'Restaurant Bar Coty y Pablo'),
 (0.9973784899851754, 'Restaurant Familiar El Chino'),
 (0.9972278979442409, 'Giovannis'),
 (0.9971292275833114, 'Kiku Cuernavaca'),
 (0.9966984353939742, 'El Oceano Dorado'),
 (0.9932641338051186, 'El cotorreo'),
 (0.9787778068873734, 'Arrachela Grill')]

#**Ejercicio-4.**

Disminuye ahora la dimensión de la factorización con un número de componentes tal, que las 
recomendaciones con esta menor cantidad de información coincidan en al menos 9 de las 10 
recomendaciones dadas con todos los valores singulares. Indica la cantidad de valores singulares 
encontrado y la cantidad de información que contienen. 
 

In [ ]:
None

#**Ejercicio-5.**

Repite el ejercicio con el restaurante “"tacos de barbacoa enfrente del Tec". Incluye tus conclusiones. 

In [ ]:
restaurantB = 'tacos de barbacoa enfrente del Tec'
tacosidx = places_list.index(restaurantB)
tacosidx

124

In [ ]:
corr_recomienda_taco = get_corr_recomendation(tacosidx)

(129,)
[ 8.26808339e-02 -1.60487509e-01  5.32570598e-01 -2.97836619e-01
 -4.73811117e-03 -7.41763626e-02  1.20101547e-01  9.66995956e-01
  9.97711045e-01  2.94148153e-01  9.44135223e-02  3.02493721e-02
  2.81438921e-01  3.07379599e-02  1.63596498e-01 -4.50130527e-02
 -3.79465414e-02  1.89886821e-01 -1.33824127e-01  2.77343149e-01
 -7.28129478e-02 -3.28427076e-02  3.18813635e-01  2.90548961e-01
  9.95809317e-01  9.98270371e-01  2.86082505e-01  1.80550369e-01
  2.93887446e-01 -3.40720748e-02  5.85428853e-01  1.36200061e-01
  3.83557567e-01  4.90786788e-01 -7.04947453e-02 -4.84781779e-01
  9.98219729e-01  6.17595183e-03 -2.20939293e-02  1.33496375e-01
 -1.29114556e-01  2.79102713e-02 -2.59011076e-03  2.59717547e-01
  3.89663781e-01  2.71522358e-02  4.70112294e-02  2.90859259e-01
  9.99520175e-01  2.94368156e-01 -2.06162660e-01 -2.22211392e-02
  4.26175353e-01  1.06394830e-01 -3.07773168e-02  6.00756809e-04
  8.77004770e-02 -3.02227119e-01  4.95320763e-02 -1.76066193e-01
  3.00827553e-02 -

In [ ]:
corr_recomienda_taco[tacosidx]

1.0

In [ ]:
idx = (corr_recomienda > .95)& (corr_recomienda < .999)
def get_corr_recomendation_places(queryidx):
  tmp = list()
  for i in range(len(places[queryidx])):
    tmp.append((corr_recomienda[queryidx][i],places[queryidx][i]))

  sorted(tmp, key=lambda x:x[0], reverse=True)

  return pd.DataFrame(tmp, columns=['Correlacion','Restaurante'])
  # recomendation_places.sort_values('Correlacion',ascending=False)

recomendation_taco_places = get_corr_recomendation_places(idx)
recomendation_taco_places

,Correlacion,Restaurante
0,0.978778,Arrachela Grill
1,0.998872,Cafeteria cenidet
2,0.996698,El Oceano Dorado
3,0.993264,El cotorreo
4,0.997228,Giovannis
5,0.997129,Kiku Cuernavaca
6,0.998008,Los Vikingos
7,0.998971,McDonalds Centro
8,0.997955,Restaurant Bar Coty y Pablo
9,0.997378,Restaurant Familiar El Chino


#**Ejercicio-6.**

Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los 
usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la 
evaluación dada a la comida (food_rating). Incluye tus conclusiones. 

In [ ]:
UtMx = tabla_relacional.pivot_table(values='food_rating', index='userID',columns='name', fill_value=0)
UtMx.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
print('Total de elementos de la matriz de utilidad: %d' %(UtMx.size))
print('Total de elementos diferentes de cero: %d' %(np.count_nonzero(UtMx)))
print('Procentaje de elementos diferentes de cero: %.1f%%'%(100*np.count_nonzero(UtMx) / UtMx.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 895
Procentaje de elementos diferentes de cero: 5.0%


In [ ]:
X = UtMx.T # Usamos la Transpuesta para cambiar los renglones
X.shape

(129, 138)

In [ ]:
# el numero de componentes debe ser menor que el numero de caracteristicas / usuarios
SVD = TruncatedSVD(n_components=137,random_state=1)
SVD.fit(X)
num_sv = 10 # 10
print('Cantidad de informacion simplificada de los primeros %d vectores singulares:'% num_sv)
print('%.1f%%' % (100 * (1-(SVD.singular_values_[0:num_sv]).sum()/(SVD.singular_values_).sum())))

Cantidad de informacion simplificada de los primeros 10 vectores singulares:
75.9%


In [ ]:
SVD.singular_values_


array([1.88543634e+01, 1.20545649e+01, 1.00731972e+01, 9.27934973e+00,
       8.95988034e+00, 8.83007545e+00, 8.81945555e+00, 8.58240932e+00,
       8.01193952e+00, 7.84658041e+00, 7.36115248e+00, 7.35619144e+00,
       7.09998786e+00, 6.87249421e+00, 6.76721627e+00, 6.51902415e+00,
       6.35901328e+00, 6.28707527e+00, 5.90794570e+00, 5.84547536e+00,
       5.77794593e+00, 5.56586970e+00, 5.52116339e+00, 5.42698527e+00,
       5.39480801e+00, 5.25745870e+00, 5.12234786e+00, 5.02365378e+00,
       5.02231066e+00, 4.89248649e+00, 4.70140189e+00, 4.68438246e+00,
       4.63438747e+00, 4.55349022e+00, 4.52469699e+00, 4.46186430e+00,
       4.34526760e+00, 4.34187458e+00, 4.32638885e+00, 4.24769259e+00,
       4.12618960e+00, 4.04231891e+00, 4.02634927e+00, 3.84124555e+00,
       3.77844237e+00, 3.69410306e+00, 3.65058383e+00, 3.64874525e+00,
       3.50456642e+00, 3.38710024e+00, 3.34267731e+00, 3.27670767e+00,
       3.20254017e+00, 3.17842606e+00, 3.17113124e+00, 3.08308895e+00,
      

In [ ]:
num_sv = 10
SVD = TruncatedSVD(n_components=num_sv,random_state=1)

resultant_matrix = SVD.fit_transform(X)
resultant_matrix

array([[ 2.15785842e-01, -5.27565692e-01,  1.39898409e+00, ...,
         1.01346312e+00,  4.72861534e-02, -1.37596363e-02],
       [ 1.82869128e-07, -6.87022175e-05, -5.08013559e-04, ...,
         3.83948880e-03,  4.76284056e-03,  6.32385517e-03],
       [ 2.98375986e+00,  1.93635474e+00, -1.96725744e-01, ...,
         9.86385788e-01,  4.28690986e-01, -1.41885805e+00],
       ...,
       [ 9.22937393e-01,  1.69605811e+00,  1.15615784e+00, ...,
        -1.06874955e+00,  5.30181778e-01,  1.13575330e-03],
       [ 8.79297757e-08, -5.32102350e-05,  1.20424560e-04, ...,
         1.92642739e-03,  1.37961302e-03, -5.99150446e-03],
       [ 1.17480154e-07, -3.54625251e-05,  1.60469823e-04, ...,
         8.23697839e-03,  4.49412814e-03,  7.14443566e-03]])

In [ ]:
CorrMx = np.corrcoef(resultant_matrix)
CorrMx.shape

(129, 129)

In [ ]:
CorrMx[0:5,0:5]

array([[ 1.        , -0.09974588, -0.19198202, -0.06041075, -0.1015953 ],
       [-0.09974588,  1.        , -0.23566704, -0.17630081,  0.9984363 ],
       [-0.19198202, -0.23566704,  1.        ,  0.10191616, -0.20764416],
       [-0.06041075, -0.17630081,  0.10191616,  1.        , -0.18507957],
       [-0.1015953 ,  0.9984363 , -0.20764416, -0.18507957,  1.        ]])

In [ ]:
restaurant = "Subway"
places = UtMx.columns 
places_list = list(places) # saber en que columna estaba Subway 
id_restaurant = places_list.index(restaurant) # busquemos el undice del restaurat Subway
id_restaurant
places

Index(['Abondance Restaurante Bar', 'Arrachela Grill', 'Cabana Huasteca',
       'Cafe Chaires', 'Cafeteria cenidet',
       'Cafeteria y Restaurant El Pacifico', 'Carls Jr',
       'Carnitas Mata  Calle 16 de Septiembre',
       'Carreton de Flautas y Migadas', 'Cenaduria El RincÃ³n de Tlaquepaque',
       ...
       'puesto de tacos', 'rockabilly ', 'shi ro ie', 'sirloin stockade',
       'tacos abi', 'tacos de barbacoa enfrente del Tec',
       'tacos de la estacion', 'tacos los volcanes', 'tortas hawai', 'vips'],
      dtype='object', name='name', length=129)

In [ ]:
corr_recomienda = get_corr_recomendation(id_restaurant)

(129,)
[-0.1018227   0.9981573  -0.20349613 -0.18639286  0.99995119 -0.05403973
  0.01553226 -0.11222985 -0.10081288 -0.03955489 -0.07706389  0.9999676
 -0.05644603 -0.10731528 -0.09854328 -0.00132181  0.99978587 -0.12081048
  0.02733706 -0.19967414  0.99948057  0.99996107 -0.09941543 -0.03072042
 -0.10775554 -0.1064965   0.04100856  0.08514219 -0.05637066  0.9999639
 -0.08070378  0.02334662 -0.03431843 -0.12750077  0.09066417 -0.17831798
 -0.10681246  0.9999959   0.99996771 -0.04213715 -0.22145722  0.99998918
  0.99996651 -0.12731476 -0.0694348   0.99999184  0.99997657 -0.06966823
 -0.10734943 -0.09699662 -0.11559105  0.99997528 -0.13017994 -0.09540099
  0.99996541  0.99997002 -0.02109808 -0.15105673  0.99999966 -0.24009039
  0.99999599 -0.00226004 -0.11580158 -0.18988177  0.99987604 -0.13094251
 -0.06655429 -0.01861152 -0.16056998 -0.30120412 -0.1951135   0.06323601
 -0.14450763  0.02220312  0.07751666 -0.09141048 -0.10564697  0.01402658
  0.02960613 -0.11702111 -0.09392049  0.064362

In [ ]:
corr_recomienda[id_restaurant]

1.0

In [ ]:
recomendation_places_by_food_rating = get_corr_recomendation_places(idx)
recomendation_places_by_food_rating

,Correlacion,Restaurante
0,0.998157,Arrachela Grill
1,0.999951,Cafeteria cenidet
2,0.999786,El Oceano Dorado
3,0.999481,El cotorreo
4,0.999961,Giovannis
5,0.999964,Kiku Cuernavaca
6,0.999968,Los Vikingos
7,0.999967,McDonalds Centro
8,0.999975,Restaurant Bar Coty y Pablo
9,0.999965,Restaurant Familiar El Chino


#**Ejercicio-7.**

Repite los ejercicios 1 a 4 generando ahora la matriz de utilidad en la cual los renglones son los 
usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la 
evaluación dada al servicio (service_rating). Incluye tus conclusiones. 

In [ ]:
UtMx = tabla_relacional.pivot_table(values='service_rating', index='userID',columns='name', fill_value=0)
UtMx.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print('Total de elementos de la matriz de utilidad: %d' %(UtMx.size))
print('Total de elementos diferentes de cero: %d' %(np.count_nonzero(UtMx)))
print('Procentaje de elementos diferentes de cero: %.1f%%'%(100*np.count_nonzero(UtMx) / UtMx.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 846
Procentaje de elementos diferentes de cero: 4.8%


In [ ]:
X = UtMx.T # Usamos la Transpuesta para cambiar los renglones
X.shape

(129, 138)

In [ ]:
# el numero de componentes debe ser menor que el numero de caracteristicas / usuarios
SVD = TruncatedSVD(n_components=137,random_state=1)
SVD.fit(X)
num_sv = 10 # 10
print('Cantidad de informacion simplificada de los primeros %d vectores singulares:'% num_sv)
print('%.1f%%' % (100 * (1-(SVD.singular_values_[0:num_sv]).sum()/(SVD.singular_values_).sum())))

Cantidad de informacion simplificada de los primeros 10 vectores singulares:
75.6%


In [ ]:
SVD.singular_values_


array([1.71371617e+01, 1.15595032e+01, 9.58637075e+00, 8.77978932e+00,
       8.67789438e+00, 8.58159980e+00, 8.15554560e+00, 7.74509335e+00,
       7.40485706e+00, 7.14037674e+00, 6.89831412e+00, 6.70453486e+00,
       6.48951690e+00, 6.37022734e+00, 6.18435929e+00, 6.05196965e+00,
       5.98224178e+00, 5.82899281e+00, 5.66161154e+00, 5.47955023e+00,
       5.20189756e+00, 5.13241484e+00, 5.12425523e+00, 5.08821407e+00,
       5.04451904e+00, 4.80243176e+00, 4.73758447e+00, 4.69249103e+00,
       4.59554226e+00, 4.53933620e+00, 4.50898682e+00, 4.34752581e+00,
       4.33282882e+00, 4.25078976e+00, 4.16881641e+00, 4.08372394e+00,
       3.96552640e+00, 3.90720970e+00, 3.87407207e+00, 3.83953149e+00,
       3.78932456e+00, 3.77429036e+00, 3.73317652e+00, 3.66288511e+00,
       3.51759865e+00, 3.43018405e+00, 3.38232186e+00, 3.35743455e+00,
       3.32745203e+00, 3.23695769e+00, 3.21576611e+00, 3.07573713e+00,
       3.04912158e+00, 2.98606817e+00, 2.93376552e+00, 2.83288658e+00,
      

In [ ]:
num_sv = 10
SVD = TruncatedSVD(n_components=num_sv,random_state=1)

resultant_matrix = SVD.fit_transform(X)
resultant_matrix

array([[ 5.02825386e-01, -8.71498158e-01,  1.84108811e+00, ...,
        -3.36466334e-01, -1.59973809e-01,  2.05846232e-01],
       [ 2.43939978e-07, -9.34635496e-05,  5.07347699e-04, ...,
        -7.27851038e-03, -4.61097492e-03, -2.18000691e-03],
       [ 2.52427474e+00,  2.18934514e+00, -7.51436196e-02, ...,
        -1.23931365e+00,  2.57458609e+00, -1.11209225e+00],
       ...,
       [ 8.75916651e-01,  1.51596081e+00,  1.62710427e+00, ...,
         1.56389284e-01, -4.43072971e-01, -1.20700643e+00],
       [ 2.67809513e-08,  2.26860063e-05,  4.39490194e-04, ...,
         6.74122842e-04,  4.54254985e-03,  3.42109485e-03],
       [-1.30393436e-07, -1.50172282e-05, -3.27644161e-04, ...,
        -1.44864120e-03, -2.65327482e-03, -4.83427108e-03]])

In [ ]:
CorrMx = np.corrcoef(resultant_matrix)
CorrMx.shape

(129, 129)

In [ ]:
CorrMx[0:5,0:5]

array([[ 1.        , -0.00843257, -0.02177244,  0.07833723, -0.01437194],
       [-0.00843257,  1.        , -0.08679474, -0.31652734,  0.99913469],
       [-0.02177244, -0.08679474,  1.        ,  0.41184087, -0.08976905],
       [ 0.07833723, -0.31652734,  0.41184087,  1.        , -0.31804555],
       [-0.01437194,  0.99913469, -0.08976905, -0.31804555,  1.        ]])

In [ ]:
restaurant = "Subway"
places = UtMx.columns 
places_list = list(places) # saber en que columna estaba Subway 
id_restaurant = places_list.index(restaurant) # busquemos el undice del restaurat Subway
id_restaurant

88

In [ ]:
corr_recomienda = get_corr_recomendation(id_restaurant)

(129,)
[-0.0178282   0.9981437  -0.09202317 -0.3227046   0.99979937 -0.07721027
 -0.05402241 -0.13599339  0.04163162 -0.03946491  0.05571452  0.99978505
  0.01533093  0.02565518 -0.04685647 -0.04911558  0.999885   -0.08279889
 -0.18920681  0.12405716  0.99948882  0.99933875 -0.24375789 -0.14505513
 -0.0471446  -0.2067366  -0.03871961  0.00242904  0.02316255  0.99955311
 -0.0275451  -0.05554585 -0.295267    0.06519893 -0.15945987 -0.00770724
 -0.03617305  0.99992609  0.99919457 -0.22503566 -0.34484697  0.99996918
  0.99990093 -0.22516002 -0.18167142  0.99977019  0.99999691  0.09134744
 -0.09058296 -0.0385161  -0.04952922  0.99967558 -0.12977223 -0.02939837
  0.99960219  0.99994703 -0.09663601  0.08812904  0.99999926  0.00560347
  0.99999752 -0.0729059   0.04167725  0.02871675  0.99979908 -0.02268788
 -0.01684029 -0.01153844 -0.18986079 -0.00135789  0.06838461 -0.09032125
 -0.02414286  0.00702674 -0.07021023  0.08746437 -0.07530019 -0.03516336
 -0.23478532  0.00786019  0.03958559 -0.0733

In [ ]:
corr_recomienda[id_restaurant]

1.0

In [ ]:
#service_rating
idx = (corr_recomienda > .95)& (corr_recomienda < .9998)

recomendation_places_service_rating = get_corr_recomendation_places(idx)
recomendation_places_service_rating

,Correlacion,Restaurante
0,0.998144,Arrachela Grill
1,0.999799,Cafeteria cenidet
2,0.999785,Chilis Cuernavaca
3,0.999489,El cotorreo
4,0.999339,Giovannis
5,0.999553,Kiku Cuernavaca
6,0.999195,Los Vikingos
7,0.999770,Mikasa
8,0.999676,Restaurant Bar Coty y Pablo
9,0.999602,Restaurant Familiar El Chino


#**Ejercicio-8.**

Incluye tus conclusiones finales. En particular indica la coincidencia o no de estos tres modelos.

In [ ]:
None

In [ ]:
recomendation_taco_places

,Correlacion,Restaurante
0,0.978778,Arrachela Grill
1,0.998872,Cafeteria cenidet
2,0.996698,El Oceano Dorado
3,0.993264,El cotorreo
4,0.997228,Giovannis
5,0.997129,Kiku Cuernavaca
6,0.998008,Los Vikingos
7,0.998971,McDonalds Centro
8,0.997955,Restaurant Bar Coty y Pablo
9,0.997378,Restaurant Familiar El Chino


In [ ]:
recomendation_places_by_food_rating

,Correlacion,Restaurante
0,0.998157,Arrachela Grill
1,0.999951,Cafeteria cenidet
2,0.999786,El Oceano Dorado
3,0.999481,El cotorreo
4,0.999961,Giovannis
5,0.999964,Kiku Cuernavaca
6,0.999968,Los Vikingos
7,0.999967,McDonalds Centro
8,0.999975,Restaurant Bar Coty y Pablo
9,0.999965,Restaurant Familiar El Chino


In [ ]:
recomendation_places_service_rating

,Correlacion,Restaurante
0,0.998144,Arrachela Grill
1,0.999799,Cafeteria cenidet
2,0.999785,Chilis Cuernavaca
3,0.999489,El cotorreo
4,0.999339,Giovannis
5,0.999553,Kiku Cuernavaca
6,0.999195,Los Vikingos
7,0.999770,Mikasa
8,0.999676,Restaurant Bar Coty y Pablo
9,0.999602,Restaurant Familiar El Chino


###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**